Load the the dataset for the from system directory

In [1]:

file = open('Annexures/eng-hindi.txt', mode='rt', encoding='utf-8')

text = file.read()#.split("\n")

file.close()


In [2]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array

Split the dataset in line and seprate english and hindi part from it

In [3]:
# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split(' ') for line in  lines]
    return pairs

Clean the dataset by removing escape character 

In [4]:
def clean_pairs(lines):
    cleaned = list()
   
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return array(cleaned)

In [5]:
text_str=to_pairs(text)

In [6]:
text_str[0]

['aadhar', 'आधार']

In [7]:
text_str[0]

['aadhar', 'आधार']

In [8]:
enghin_pairs = clean_pairs(text_str)

In [9]:
print(enghin_pairs[9][1])
print(enghin_pairs[9])

ए ब े क स
['a b a c u s', 'ए ब े क स']


In [10]:
#len(enghin_pairs)

In [11]:
englis=[]
hinlis=[]

In [12]:
for i in range(len(enghin_pairs)):
    englis.append(enghin_pairs[i][0])
    hinlis.append(enghin_pairs[i][1])
    #print('[%s] => [%s]' % (enghin_pairs[i][0], enghin_pairs[i] [1]))

In [13]:
englis[0]

'a a d h a r'

In [14]:
len(text_str)

6999

Train Test split of english and hindi part of dataset

In [15]:
train,test=enghin_pairs[:5500],enghin_pairs[5500:]

In [16]:
trainen,testen=englis[:5500],englis[5500:]
trainhin,testhin=hinlis[:5500],hinlis[5500:]

In [17]:
(enghin_pairs[:5500][5])

['a b a c i n a t e', 'अ ब े स ि न े ट']

In [18]:
train[0][1]

'आ ध ा र'

Tokenize the the text using keras tokenizer on the basis of character count

In [19]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

Find the max length character from corpus 

In [20]:

# max sentence length
def max_length(lines):
    return max(len(line.split()) for line in lines)

In [21]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [22]:
enghin_pairs[0:] [0]

['a a d h a r', 'आ ध ा र']

In [23]:
hinlis

['आ ध ा र',
 'आ ज़ म ग ढ़',
 'आ ज ़ म ा य ि श',
 'अ ब ा ब ा',
 'अ ब े स ि न े ट े ड',
 'अ ब े स ि न े ट',
 'अ ब ै क',
 'अ ब ै क ् ट ि न ल',
 'अ ब ै क स',
 'ए ब े क स',
 'अ ब द ा न',
 'अ ब ा द ा',
 'आ ब ा द',
 'अ ब ै फ ़ ् ट',
 'अ ब ल ा',
 'ए ब ल ो न ् स',
 'ए ब ल ो न',
 'ऐ ब े ं प ि य र',
 'ऐ ब े म ् प',
 'अ ब ै ं ड न ् ड',
 'अ ब ै ं ड न ि ं ग',
 'अ ब ै ं ड न म े न ् ट',
 'अ ब ै ं ड न ् स',
 'अ ब ै ं ड न',
 'अ ब न ी न ् द ् र न ा थ',
 'अ व न ी न ् द ् र न ा थ',
 'अ ब न ी',
 'अ ब ा न',
 'ए ब ी ए प ी',
 'अ ब र ् ट ि क ु ल े श न',
 'आ ब ा स ा ह े ब',
 'अ ब े स म े ं ट',
 'अ ब े ज़',
 'अ ब ै श ् ड',
 'अ ब ै श म े न ् ट',
 'अ ब ै श',
 'अ ब ै श ़ ् य ा',
 'अ ब े स ि क',
 'अ ब े ट े ब ल',
 'अ ब े ट े ड',
 'अ ब े ट म े ं ट',
 'अ ब े ट ् स',
 'अ ब े ट',
 'अ ब े ट ि क',
 'अ ब े ट ि ं ग',
 'अ ब े ट ि स',
 'अ ब े ट र',
 'अ ब त ् त ि स',
 'ए ब े ट ो इ र',
 'अ ब ै क ् स ि य ा ल ि',
 'अ ब ै क ् स ि य ा ल',
 'अ ब ा य ा',
 'अ ब ् ब ा ए ब ् ब ा',
 'ए ब ् ब स ी',
 'अ ब ् ब ा स ि द',
 'अ ब ् ब ा स ि य ो ं',


Tokenize english and hindi Vocabulary

In [24]:

eng_tokenizer = create_tokenizer(englis)
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(englis)


print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
hin_tokenizer = create_tokenizer(hinlis)
hin_vocab_size = len(hin_tokenizer.word_index) + 1
hin_length = max_length(hinlis)
print('Hindi Vocabulary Size: %d' % hin_vocab_size)
print('Hindi Max Length: %d' % (hin_length))

English Vocabulary Size: 31
English Max Length: 21
Hindi Vocabulary Size: 69
Hindi Max Length: 21


In [25]:
#eng_tokenizer

In [26]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import  to_categorical
from numpy import array
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint

Encode the sequence of the text and pad  that to max lenght pf the corpus

In [27]:
def encode_sequences(tokenizer, length, lines):
# integer encode sequences
    X = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

Encode the label into categorical from so that categorical cross entropy loss function on it

In [28]:
# one hot encode target sequence
def encode_output(sequences, vocab_size):
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
        y = array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

Model Building using embedding layer LSTM unit with Repeat Vector layer and Time Distrbuted layer softmax as a activation function 

In [29]:
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    model = Sequential()
    model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(tar_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
    return model
 

Prepare training and testing data and compile the model with categorical cross entropy loss and adam optimizer

In [30]:

eng_tokenizer = create_tokenizer(englis)
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(englis)
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))

hin_tokenizer = create_tokenizer(hinlis)
hin_vocab_size = len(hin_tokenizer.word_index) + 1
hin_length = max_length(hinlis)
print('Hindi Vocabulary Size: %d' % hin_vocab_size)
print('Hindi Max Length: %d' % (hin_length))
 
# prepare training data
trainX = encode_sequences(hin_tokenizer, eng_length, trainen)
trainY = encode_sequences(eng_tokenizer,hin_length , trainhin)
trainY = encode_output(trainY, hin_vocab_size)
# prepare validation data
testX = encode_sequences(hin_tokenizer, eng_length, testen)
testY = encode_sequences(eng_tokenizer, hin_length, testhin)
testY = encode_output(testY, hin_vocab_size)
 
# define model
model = define_model(eng_vocab_size, hin_vocab_size, eng_length, hin_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

English Vocabulary Size: 31
English Max Length: 21
Hindi Vocabulary Size: 69
Hindi Max Length: 21


Train the model and store the weights in h5 format

In [31]:
filename = 'model.h5'

In [32]:

#checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
#model.fit(trainX, trainY, epochs=150, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)

Train on 5500 samples, validate on 1499 samples
Epoch 1/150
 - 6s - loss: 0.5869 - acc: 1.0000 - val_loss: 0.1103 - val_acc: 0.9999

Epoch 00001: val_loss improved from inf to 0.11035, saving model to model.h5
Epoch 2/150
 - 4s - loss: 0.0109 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00002: val_loss improved from 0.11035 to 0.00000, saving model to model.h5
Epoch 3/150
 - 4s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00003: val_loss did not improve from 0.00000
Epoch 4/150
 - 4s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00004: val_loss did not improve from 0.00000
Epoch 5/150
 - 3s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00005: val_loss did not improve from 0.00000
Epoch 6/150
 - 3s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00006: val_loss did not improve from 0.00000
Epoch 7/150
 - 4s - loss: 1.1921e-07 - acc: 

 - 3s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00057: val_loss did not improve from 0.00000
Epoch 58/150
 - 3s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00058: val_loss did not improve from 0.00000
Epoch 59/150
 - 3s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00059: val_loss did not improve from 0.00000
Epoch 60/150
 - 3s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00060: val_loss did not improve from 0.00000
Epoch 61/150
 - 4s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00061: val_loss did not improve from 0.00000
Epoch 62/150
 - 4s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00062: val_loss did not improve from 0.00000
Epoch 63/150
 - 4s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00063: val_loss did not improve from 0.0000


Epoch 00113: val_loss did not improve from 0.00000
Epoch 114/150
 - 3s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00114: val_loss did not improve from 0.00000
Epoch 115/150
 - 3s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00115: val_loss did not improve from 0.00000
Epoch 116/150
 - 3s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00116: val_loss did not improve from 0.00000
Epoch 117/150
 - 3s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00117: val_loss did not improve from 0.00000
Epoch 118/150
 - 3s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00118: val_loss did not improve from 0.00000
Epoch 119/150
 - 3s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 0.9999

Epoch 00119: val_loss did not improve from 0.00000
Epoch 120/150
 - 3s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e

In [33]:
from keras.models import load_model

In [34]:
model = load_model(filename)

In [35]:
score = model.evaluate(testX,testY,verbose=0)
#print(testY[0])
model.metrics_names

['loss', 'acc']

Accuracy measure of the model

In [36]:
print("The accuracy of this model on Test set is",score[1])

The accuracy of this model on Test set is 0.999904698733889


In [54]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 21, 256)           7936      
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 21, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 21, 256)           525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 21, 69)            17733     
Total params: 1,076,293
Trainable params: 1,076,293
Non-trainable params: 0
_________________________________________________________________


<font size=5><b>Future Improvement </b></font><br/>
<font size=3>
Currently from the learning curve I can say that  the model is getting overfitting the possible measure we can do improve the model
<ol>
  <li>Use drop for the regulization purpose</li>
  <li>Add Noise to the dataset</li>
  <li>Use Fast.ai api for the embedding of Hindi and english words</li>
  <li>Use GAN architecure with two categorical loss function (if we high volume of data)</li>
  <li>Use advance BERT model to achive this task</li>
  <li>Most obivous increase the training data</li>
  <li>Use attention model only(if we high volume of data)</li>
  
</ol> 
</font>